## Scrape cars on sale on various trader websites

Approach is different for this one. All of the trader's car can be listed on one page. I've simply loaded the entire page and saved it, with picture files. This is what I'm parsing. 

This could be set up to be done on a regular basis to identify new cars, sold cars, and so on. 

First saved file on 09/12/2016.

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sqlite3
import time

%load_ext sql
%config SqlMagic.autopandas=True
%matplotlib inline

/usr/local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [ ]:
# %qtconsole

In [ ]:
# Connect to dbs
conn = sqlite3.connect('data/sales.db')
%sql sqlite:///data/zipcode.sqlite

In [ ]:
#Get list of zipcodes, only do TX for now (LIMIT 10 because super slow... 
zips = %sql select ZipCode from zipcode where St = 'TX' limit 10

In [ ]:
trader = 'http://www.autotrader.com'



In [ ]:
for z in zips.ZipCode:
    
    print(z)

    browser = webdriver.Chrome()
    browser.get(trader)

    elem = browser.find_element_by_name('fyc-form-j_id_bb-j_id_bn-j_id_cp-zipcode')  # Find the search box
    elem.send_keys(Keys.ESCAPE)
    elem.send_keys(str(z))
    elem.send_keys(Keys.RETURN)
    time.sleep(20)    

    not_end = 0
    while not_end < 4:
        soup = BeautifulSoup(browser.page_source, "lxml")

        # Create DataFrame
        # columns = ['NewUsed', 'Year', 'Make', 'Type','Mileage','Price','ZipCode','Source']
        all_cars = pd.DataFrame() #columns=columns)

        #Make and Type
        brands = soup.find_all("span", class_="atcui-truncate ymm")
        #Price and Mileage
        details = soup.find_all("div", class_="atcui-section atcui-clearfix   listing-content ")

        for cars, entry in zip(brands, details):
            carsSTR = cars.text.split()
            car_nu = carsSTR[0]
            car_year = int(carsSTR[1])
            car_make = carsSTR[2]
            car_type = " ".join(carsSTR[3:])

            try:
                milesSTR = entry.find_all("span", class_="mileage")[0].text
                car_miles = int(milesSTR.split()[0].replace(',',''))
            except:
                car_miles = 'Null'

            try:
                priceSTR = entry.find_all("h4", class_="primary-price")[0].text
                car_price = int(priceSTR.replace('$','').replace(',',''))
            except:
                car_price = 'Null'


    #         print('{} {} {} {} {} {}'.format(car_year, car_nu, car_make, car_type, car_price, car_miles))
            all_cars = all_cars.append(pd.DataFrame([[car_year, car_nu, car_make, 
                                                      car_type, car_miles, car_price, 
                                                      z, trader]]),
                                      ignore_index = True)

        all_cars.columns = ['new_used', 'year', 'make', 'type', 'mileage', 'price', 'zipcode', 'source']    
        # Update index
        index_max = conn.execute('SELECT MAX(car_id) FROM sales').fetchall()
        if index_max[0][0] is not None:
            all_cars.index = all_cars.index + 1 + index_max[0][0]
        all_cars.to_sql('sales',conn,flavor='sqlite',if_exists='append',index_label='car_id')

        next_counter = 0
        while next_counter < 4:
            try: 
                browser.find_element_by_class_name('pagination-button-next').click()
                next_counter = 4
            except:
                if next_counter == 3:
                    not_end = 1
                time.sleep(2)
                next_counter = next_counter + 1

    browser.quit()






In [ ]:
conn.close()